### Training on more temporal data is better. 
If it is not better with default split, try a fixed split. If it is not better with that try gradually sampling older data less. 

In [ ]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
from utils.ipynb_helpers import setting_from_args, handle_gpu

In [ ]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = './data/ETT/' # root path of data file


args.data_path = 'close.csv' # data file
args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'XOM_close' # target feature in S or MS task
args.freq = 't' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './checkpoints' # location of model checkpoints

# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.c_out = 1 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0

args.batch_size = 32 
args.learning_rate = 0.00001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1 # number of runs
args.train_epochs = 10
args.patience = 4
args.des = 'assumption_time'

args.inverse = True # Defaultly False but @Zac thinks it should be True

handle_gpu(args, None)

# idk what this is for
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [ ]:
args.seq_len = 128 # input sequence length of Informer encoder
args.label_len = 64 # start token length of Informer decoder
args.pred_len = 16 # prediction sequence length

args.cols = [args.target]
args.enc_in = 1 # encoder input size
args.dec_in = 1 # decoder input size


# Test data is always just the data after this date
args.date_test = "2022-04-01"

exp = None
setting = None

# "2022-01-01", 
date_cutoffs = ["2021-01-01", "2020-01-01", "2018-01-01", None]
ii = 0 
for date_cutoff in date_cutoffs:
    
    args.date_cutoff = date_cutoff

    print('Args in experiment:')
    print(args)

    Exp = Exp_Informer
    for _ in range(args.itr):
        # setting record of experiments
        setting = setting_from_args(args, ii)
        
        # set experiments
        exp = Exp(args)
        
        # train
        print(f"=======date_cutoff : {args.date_cutoff}=====================")
        print(f">>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>")
        exp.train(setting)
        
        # test
        print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
        exp.test(setting)

        torch.cuda.empty_cache()
        ii += 1